In [3]:
import pandas as pd
import numpy as np
from pandas import DataFrame, Series
import seaborn as sb
import matplotlib.pyplot as plt
import scipy.stats as ss


pd.set_option('display.max_colwidth', None)

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
sb.set(style="ticks", color_codes=True)

In [4]:
#Connect Google drive to colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
raw_data_path = '/content/drive/MyDrive/Colab Notebooks/workhuman/processed_FAQs.csv'
data = pd.read_csv(raw_data_path)

In [6]:
data.head()

,category,question_id,question,answer,faq_url
0,Order,55bf40a7d000003d0011ffa1,i missed the delivery of my order today. what should i do?,the courier service delivering your order usually tries to deliver on the next business day in case you miss a delivery.\nyou can check your sms for more details on when the courier service will try to deliver again.\n,https://www.flipkart.com/helpcentre?catalog=55c9cf49b0000024002c1703&faq=55bf40a7d000003d0011ffa1&view=FAQ
1,Order,55f2a6eb5b00001e003202ed,will the delivery be tried again if i'm not able to collect my order the first time?,couriers make sure that the delivery is re-attempted the next working day if you can't collect your order the first time.,https://www.flipkart.com/helpcentre?catalog=55c9cf49b0000024002c1703&faq=55f2a6eb5b00001e003202ed&view=FAQ
2,Order,55f2a6995b00002b003202be,the delivery of my order is delayed. what should i do?,"on the rare occasion that your order is delayed, please check your email & messages for updates. a new delivery timeframe will be shared with you and you can also track its status by visiting my orders.\n",https://www.flipkart.com/helpcentre?catalog=55c9cf49b0000024002c1703&faq=55f2a6995b00002b003202be&view=FAQ
3,Order,55f2a53f5c00003200342b9c,what should i do if my order is approved but hasn't been shipped yet?,sellers usually ship orders 1-2 business days before the delivery date so that they reach you on time. in case your order hasn't been shipped within this time please contact our customer support so that we can look into it.,https://www.flipkart.com/helpcentre?catalog=55c9cf49b0000024002c1703&faq=55f2a53f5c00003200342b9c&view=FAQ
4,Order,55e9455f5a00002500d5e024,can i take the shipment after opening and checking the contents inside?,"as per company policy, a shipment can't be opened before delivery, but you can accept the shipment and get in touch with us later in case you have any concerns.",https://www.flipkart.com/helpcentre?catalog=55c9cf49b0000024002c1703&faq=55e9455f5a00002500d5e024&view=FAQ


In [6]:
!pip install datasets sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (4

In [1]:
import sys
import argparse
import pandas as pd
import numpy as np
from datasets import Dataset
from sentence_transformers import SentenceTransformer, losses, util, InputExample
# from sklearn.metrics.pairwise import cosine_similarity
from torch.utils.data import DataLoader
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score


In [8]:
df = data
questions = df['question'].tolist()
answers = df['answer'].tolist()

model_name = "all-MiniLM-L6-v2"
model = SentenceTransformer(model_name)

# Prepare inputExample objects
examples = []
for q, a in zip(questions, answers):
    examples.append(InputExample(texts=[q, a]))

# Define the loss function
train_loss = losses.MultipleNegativesRankingLoss(model)


# Define k-fold cross validation
k = 5
kf = KFold(n_splits=k, shuffle=True)
fold_count = 0

accuracies = []
precisions = []
recalls = []

for train_index, val_index in kf.split(examples):
    fold_count += 1
    print(f"Fold {fold_count}:")

    train_examples = [examples[i] for i in train_index]
    val_examples = [examples[i] for i in val_index]

    train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
    val_dataloader = DataLoader(val_examples, shuffle=True, batch_size=16)

    # Define the loss function
    train_loss = losses.MultipleNegativesRankingLoss(model)


    # Fine-tune the model
    model.fit(
        train_objectives=[(train_dataloader, train_loss)],
        epochs=1,
        warmup_steps=100,
        show_progress_bar=True
    )

    # Evaluate the model on validation set
    val_questions = [example.texts[0] for example in val_examples]
    val_answers = [example.texts[1] for example in val_examples]

    val_embeddings = model.encode(val_questions, convert_to_tensor=True)
    question_embeddings = model.encode(questions, convert_to_tensor=True)

    y_true = []
    y_pred = []

    for i, example in enumerate(val_examples):
        cos_scores = util.pytorch_cos_sim(val_embeddings[i], question_embeddings)[0]
        best_match_idx = cos_scores.argmax().item()
        predicted_answer = answers[best_match_idx]
        actual_answer = example.texts[1]

        y_true.append(actual_answer)
        y_pred.append(predicted_answer)

        print(f"Question: {example.texts[0]}")
        print(f"Predicted Answer: {predicted_answer}")
        print(f"Actual Answer: {actual_answer}")
        print("-" * 50)

    # Convert answers to binary format for calculating precision and recall
    y_true_binary = [1 if a == p else 0 for a, p in zip(val_answers, y_pred)]
    y_pred_binary = [1 for _ in y_pred]  # All predictions are considered as 1

    # Calculate accuracy, precision, and recall
    accuracy = accuracy_score(y_true_binary, y_pred_binary)
    precision = precision_score(y_true_binary, y_pred_binary, zero_division=0)
    recall = recall_score(y_true_binary, y_pred_binary, zero_division=0)

    accuracies.append(accuracy)
    precisions.append(precision)
    recalls.append(recall)

# Calculate mean accuracy, precision, and recall
mean_accuracy = np.mean(accuracies)
mean_precision = np.mean(precisions)
mean_recall = np.mean(recalls)

print(f"Mean Accuracy: {mean_accuracy}")
print(f"Mean Precision: {mean_precision}")
print(f"Mean Recall: {mean_recall}")


Fold 1:


Step,Training Loss


Question: the delivery of my order is delayed. what should i do?
Predicted Answer: on the rare occasion that your order is delayed, please check your email & messages for updates. a new delivery timeframe will be shared with you and you can also track its status by visiting my orders.
 
Actual Answer: on the rare occasion that your order is delayed, please check your email & messages for updates. a new delivery timeframe will be shared with you and you can also track its status by visiting my orders.
 
--------------------------------------------------
Question: how quickly can i get my order delivered?
Predicted Answer: orders will be delivered by the date you see on the product page for your location. 
Actual Answer: orders will be delivered by the date you see on the product page for your location. 
--------------------------------------------------
Question: what should i do if i don't get the invoice for my order?
Predicted Answer: sellers send a hard copy of the invoice in the sh

Step,Training Loss


Question: i missed the delivery of my order today. what should i do?
Predicted Answer: the courier service delivering your order usually tries to deliver on the next business day in case you miss a delivery.
you can check your sms for more details on when the courier service will try to deliver again.
 
Actual Answer: the courier service delivering your order usually tries to deliver on the next business day in case you miss a delivery.
you can check your sms for more details on when the courier service will try to deliver again.
 
--------------------------------------------------
Question: my order has reached the nearest delivery hub, but why isn't it out for delivery yet?
Predicted Answer: the courier service delivering your order are responsible for making sure that your order reaches you within the delivery date. rest assured, you'll get an sms once your shipment is out for delivery.
Actual Answer: the courier service delivering your order are responsible for making sure that you

...

Question: what should i do if my order is approved but hasn't been shipped yet?
Predicted Answer: sellers usually ship orders 1-2 business days before the delivery date so that they reach you on time. in case your order hasn't been shipped within this time please contact our customer support so that we can look into it.
Actual Answer: sellers usually ship orders 1-2 business days before the delivery date so that they reach you on time. in case your order hasn't been shipped within this time please contact our customer support so that we can look into it.
--------------------------------------------------
Question: how do i know my order has been confirmed?
Predicted Answer: an e-mail & sms will be sent once you've successfully placed your order. we'll also let you know as soon as the seller ships the item(s) to you along with the tracking number(s) for your shipment(s). you can track your orders from the 'my orders' section on your flipkart account. \n 
Actual Answer: an e-mail & sms w

Step,Training Loss


Question: my order status shows 'out for delivery', but i haven't got it yet. when will i get it?
Predicted Answer: the courier service will contact you for delivery of your order. please check your sms for more details.
Actual Answer: the courier service will contact you for delivery of your order. please check your sms for more details.
--------------------------------------------------
Question: what is the buyer protection policy?
Predicted Answer: the buyer protection policy mediates buyer-seller disputes. in case a seller declines your request for a return of an item and you are not convinced with the reason given, you can write to us at resolution@flipkart.com for buyer protection. you can dispute the resolution that the seller has shared for your issue until 45 days from the date of delivery and concern is looked into by us on a case-to-case basis.
Actual Answer: the buyer protection policy mediates buyer-seller disputes. in case a seller declines your request for a return of a

Step,Training Loss


Question: will the delivery be tried again if i'm not able to collect my order the first time?
Predicted Answer: couriers make sure that the delivery is re-attempted the next working day if you can't collect your order the first time.
Actual Answer: couriers make sure that the delivery is re-attempted the next working day if you can't collect your order the first time.
--------------------------------------------------
Question: can i take the shipment after opening and checking the contents inside?
Predicted Answer: as per company policy, a shipment can't be opened before delivery, but you can accept the shipment and get in touch with us later in case you have any concerns.
Actual Answer: as per company policy, a shipment can't be opened before delivery, but you can accept the shipment and get in touch with us later in case you have any concerns.
--------------------------------------------------
Question: will my product be installed by the brand or flipkart? if it is installed by fl